In [60]:
import tools
import tensorflow as tf
import numpy as np
import cv2

In [61]:
def prepare_dataset(ds):
    return ds.map(
            lambda img, skel: (tf.image.resize(img, tf.constant([128, 128], dtype=tf.dtypes.int32)), skel),
            num_parallel_calls=tf.data.experimental.AUTOTUNE).map(
            lambda img, skel: (tf.cast(img, dtype=tf.float32) / tf.constant(5000.0, dtype=tf.float32),
                               (skel + tf.constant(750.0, dtype=tf.float32)) / tf.constant(1500.0, dtype=tf.float32)))


def augment_pose_dataset(ds):
    ds = ds.map(tools.pose_augmentation.flip_randomly, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return ds


In [62]:
pose_batch_size=128

In [63]:
dataset_source = 'MSRA'

pose_train_data_dir = {
        'FHAD': r"E:\MasterDaten\Datasets\FHAD",
        'NYU': r"G:\master_thesis_data\Datasets\nyu\nyu_hand_dataset_v2\dataset",
        'MSRA': r"E:\MasterDaten\Datasets\cvpr15_MSRAHandGestureDB"
        }
pose_train_data_dir = pose_train_data_dir[dataset_source]

if dataset_source == 'NYU':
    from tools import NYU as selected_dataset
elif dataset_source == 'FHAD':
    from tools import FHAD as selected_dataset
elif dataset_source == 'MSRA':
    from tools import MSRA as selected_dataset
    
_,_,hand_pose_dataset = selected_dataset.get_dataset(pose_train_data_dir)
hand_pose_dataset = augment_pose_dataset(hand_pose_dataset)
hand_pose_dataset = prepare_dataset(hand_pose_dataset)
hand_pose_dataset = hand_pose_dataset.shuffle(pose_batch_size * 10)
hand_pose_dataset = hand_pose_dataset.batch(batch_size=pose_batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [64]:
img_max = 0
img_min = 0
skel_max = 0
skel_min = 0
img_mean = 0
skel_mean = 0
cnt = 1

for img, skel in hand_pose_dataset:
    img = img.numpy()
    skel = skel.numpy()
    
    img_max_curr = img.max()
    img_min_curr = img.min()
    skel_max_curr = skel.max()
    skel_min_curr = skel.min()
    
    if img_max_curr > img_max:
        img_max = img_max_curr
    if img_min_curr < img_min:
        img_min = img_min_curr
        
    if skel_max_curr > skel_max:
        skel_max = skel_max_curr
    if skel_min_curr < skel_min:
        skel_min = skel_min_curr
        
    skel_mean = ((cnt-1) * skel_mean + skel.mean()) / cnt
    img_mean = ((cnt-1) * img_mean + img.mean()) / cnt

    cnt += 1
    
    if cnt % 100 == 0:
        print("Count: {}  img_max: {}  img_min: {}  img_mean: {}  skel_max: {}  skel_min: {}  skel_mean: {}".format(
                cnt, img_max, img_min, img_mean, skel_max, skel_min, skel_mean
        ))


print("Count: {}  img_max: {}  img_min: {}  img_mean: {}  skel_max: {}  skel_min: {}  skel_mean: {}".format(
        cnt, img_max, img_min, img_mean, skel_max, skel_min, skel_mean
))

Count: 100  img_max: 0.10615000128746033  img_min: 0  img_mean: 0.0033903351862623232  skel_max: 0.6247473359107971  skel_min: 0  skel_mean: 0.42843325872613924
Count: 200  img_max: 0.15527500212192535  img_min: 0  img_mean: 0.0032214938754206476  skel_max: 0.6903733611106873  skel_min: 0  skel_mean: 0.42277987683238705
Count: 300  img_max: 0.15527500212192535  img_min: 0  img_mean: 0.0030286661948834273  skel_max: 0.6903733611106873  skel_min: 0  skel_mean: 0.419412275122161
Count: 399  img_max: 0.15527500212192535  img_min: 0  img_mean: 0.003044600284953213  skel_max: 0.6903733611106873  skel_min: 0  skel_mean: 0.42203570071177277
